# Dataset

The data is related with telephonic marketing campaigns of a Portuguese banking institution. The classification goal is to predict whether the client will subscribe a term deposit (variable y).

Source: <a href="https://www.kaggle.com/prakharrathi25/banking-dataset-marketing-targets?select=test.csv">Bank Marketing Data Set</a>

| Columns | Description | Type | Remarks |
| :- | :- | :- | :- |
| age | numeric | bank client data | |
| job | type of job | bank client data | |
| marital | marital status | bank client data | |
| education | education tier | bank client data | |
| default | has credit in default? | bank client data | |
| balance | average yearly balance, in euros | bank client data | |
| housing | has housing loan? | bank client data | |
| loan | has personal loan? | bank client data | |
| contact | contact communication type | related with last contact of the current campaign | |
| day | last contact day of the month | related with last contact of the current campaign | |
| month | last contact month of year | related with last contact of the current campaign | |
| duration | last contact duration, in seconds | related with last contact of the current campaign | |
| campaign | number of contacts performed during this campaign and for this client | others | |
| pdays | number of days that passed by after the client was last contacted from a previous campaign | others | -1 means client was not previously contacted | |
| previous | number of contacts performed before this campaign and for this client | others | | 
| poutcome | outcome of the previous marketing campaign | others | |
| y | has the target subscribed a term deposit? | target variable | |

# Conclusion from EDA Homework

1. Tidak terdapat missing values, tetapi terdapat unknown values pada beberapa feature
2. Sepertinya tidak ada issue yang mencolok pada tipe data untuk setiap kolom (sudah sesuai)
3. y adalah Feature Target
4. Tidak ada dataset yang Duplikat
5. Hampir seluruh kolom memiliki outliers
6. Tampak hampir semua feature berdistribusi *skewed*, kecuali `day`
7. Feature `day` tidak memiliki outliers karena menunjukkan tanggal pada bulan, sehingga nilainya dari 1 s/d 31
8. Column targeted terindikasi imabalance


# 1. Data Cleansing

## a) Handle Missing Value

Tidak perlu dilakukan handling, karena tidak ada **'Missing Value'** pada dataset ini

## b) Handle Duplicated Data

Tidak perlu dilakukan handling, karena tidak ada **'Duplicated Data'** pada dataset ini

## c) Handle Outliers

### Pada saat training model ada 2 skenario untuk melihat mana performa model yang lebih baik:

### 1) Dilakukan Handling

In [14]:
# Import Library
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler # standarization, normalization

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() #label encoding

from imblearn import under_sampling, over_sampling #handling imbalance data

pd.set_option('display.max_columns', None)

In [15]:
df = pd.read_excel('train.xlsx')

In [16]:
num = list(df.select_dtypes("number").columns)
cat = list(df.select_dtypes("object").columns)

In [17]:
# Handling Outlier with IQR
print(f'Jumlah baris sebelum memfilter outlier: {df.shape[0]}')

iqr_entries = np.array([True] * len(df))

for col in num:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    low_lim = q1 - (1.5 * iqr)
    high_lim = q3 + (1.5 * iqr)
    iqr_entries = ((df[col] >= low_lim) & (df[col] <= high_lim)) # filtering with IQR
    
dfi = df[iqr_entries] # filter, cuma ambil yang z-scorenya dibawah 3

print(f'Jumlah baris setelah memfilter outlier: {dfi.shape[0]}')

Jumlah baris sebelum memfilter outlier: 45211
Jumlah baris setelah memfilter outlier: 36954


In [19]:
# Handling Outlier with Zscore
print(f'Number of rows before outlier handling: {df.shape[0]}')

zscore_entries = np.array([True] * len(df))

for col in num:
    zscore = abs(stats.zscore(df[col])) # hitung absolute z-scorenya
    zscore_entries = (zscore < 3) & zscore_entries  # filtering with absolute z-score value < 3
    
dfz = df[zscore_entries] # filter, cuma ambil yang z-scorenya dibawah 3

print(f'Number of rows before outlier handling: {dfz.shape[0]}')

Number of rows before outlier handling: 45211
Number of rows before outlier handling: 40209


### 2) Tanpa dilakukan Handling

## d) Handle Transformation

#### Normalization of `duration` feature

In [22]:
df['duration_norm'] = MinMaxScaler().fit_transform(df['duration'].values.reshape(len(df), 1))
df.drop(columns=['duration'], inplace=True)

In [23]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,duration_norm
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,1,-1,0,unknown,no,0.053070
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,1,-1,0,unknown,no,0.030704
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,1,-1,0,unknown,no,0.015453
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,1,-1,0,unknown,no,0.018707
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,1,-1,0,unknown,no,0.040260


#### Normalization of `age` feature

In [30]:
df['age_norm'] = MinMaxScaler().fit_transform(df['age'].values.reshape(len(df),1))

In [31]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,duration_norm,age_norm
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,1,-1,0,unknown,no,0.053070,0.519481
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,1,-1,0,unknown,no,0.030704,0.337662
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,1,-1,0,unknown,no,0.015453,0.194805
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,1,-1,0,unknown,no,0.018707,0.376623
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,1,-1,0,unknown,no,0.040260,0.194805


## e) Feature Encoding

In [32]:
cats_label = ['education','default','housing','loan','y']

In [40]:
for cols in cats_label:
    df[cols] = label_encoder.fit_transform(df[cols])

In [41]:
df.head()

,age,education,default,balance,housing,loan,day,month,campaign,pdays,previous,y,duration_norm,age_norm,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,poutcome
0,58,2,0,2143,1,0,5,may,1,-1,0,0,0.053070,0.519481,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
1,44,1,0,29,1,0,5,may,1,-1,0,0,0.030704,0.337662,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0
2,33,1,0,2,1,1,5,may,1,-1,0,0,0.015453,0.194805,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
3,47,3,0,1506,1,0,5,may,1,-1,0,0,0.018707,0.376623,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
4,33,3,0,1,0,0,5,may,1,-1,0,0,0.040260,0.194805,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0


In [35]:
# OHE
for col in ['job', 'marital', 'contact', 'poutcome']:
    onehots = pd.get_dummies(df[col], prefix=col)
    df = df.join(onehots)

In [36]:
# drop  columns before OHE
df.drop(columns=['job','marital','contact','poutcome'], inplace=True)

In [42]:
df.head()

,age,education,default,balance,housing,loan,day,month,campaign,pdays,previous,y,duration_norm,age_norm,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,poutcome
0,58,2,0,2143,1,0,5,may,1,-1,0,0,0.053070,0.519481,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
1,44,1,0,29,1,0,5,may,1,-1,0,0,0.030704,0.337662,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0
2,33,1,0,2,1,1,5,may,1,-1,0,0,0.015453,0.194805,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
3,47,3,0,1506,1,0,5,may,1,-1,0,0,0.018707,0.376623,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
4,33,3,0,1,0,0,5,may,1,-1,0,0,0.040260,0.194805,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0


## f) Handle Class Imbalance

In [43]:
df['y'].value_counts()

0    39922
1     5289
Name: y, dtype: int64

In [46]:
X = df.drop(columns=['y'])
y = df['y']
print(X.shape)
print(y.shape)

(45211, 36)
(45211,)


In [48]:
X_under, y_under = under_sampling.RandomUnderSampler(0.5).fit_resample(X, y)
X_over, y_over = over_sampling.RandomOverSampler(0.5).fit_resample(X, y)

D:\Python\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(
D:\Python\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [49]:
print('Original')
print(pd.Series(y).value_counts())
print('UNDERSAMPLING')
print(pd.Series(y_under).value_counts())
print('OVERSAMPLING')
print(pd.Series(y_over).value_counts())

Original
0    39922
1     5289
Name: y, dtype: int64
UNDERSAMPLING
0    10578
1     5289
Name: y, dtype: int64
OVERSAMPLING
0    39922
1    19961
Name: y, dtype: int64


# 2. Feature Engineering

## a) Feature Selection

Tidak ada feature yang dibuang pada saat modelling

## b) Feature Extraction

Tidak ada feature yang perlu ditambahkan

## c) Feature Addition (Not for Implementation)

- Income
- Children/Dependants
- Location (certain city may have more user)